Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
val NAME = ""
val COLLABORATORS = ""

---

# Lab 6 - Advanced Memory
> Labs will be due each week before the homeworks. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language. Labs are autograded so you can get quick feedback.

> In this lab, you will be working your way up to building a multi-ported memory generator. It will be parameterized on the memory details (capacity & word size) as well as the number and type of ports (e.g. read or write). In the problems, you will work your way from defining parameters and interfaces to finally implementing the generator.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
val path = System.getProperty("user.dir") + "/resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chisel3.tester.RawTester.test

## Problem 1 (5 pts) - MemParams
> Write a case class `MemParams` that will be used to parameterize a memory module. Derive the case class's fields and their types from the supplied tests. Assume that if there are both read and write ports, there will be the same number of each.

In [ ]:
// YOUR CODE HERE
???

In [ ]:
val p0 = MemParams(32, 64, 1, 0)
assert(p0.about == "2048b capacity, 1 read port(s), 0 write port(s)")
assert(p0.numWords == 32)
assert(p0.wordSize == 64)
assert(p0.totalPorts == 1)
assert(p0.addrWidth == 5)

val p1 = MemParams(1024, 64, 0, 4)
assert(p1.about == "65536b capacity, 0 read port(s), 4 write port(s)")
assert(p1.numWords == 1024)
assert(p1.wordSize == 64)
assert(p1.totalPorts == 4)
assert(p1.addrWidth == 10)

val p2 = MemParams(16, 32, 2, 2)
assert(p2.about == "512b capacity, 2 read port(s), 2 write port(s)")
assert(p2.numWords == 16)
assert(p2.wordSize == 32)
assert(p2.totalPorts == 4)
assert(p2.addrWidth == 4)

## Problem 2 (3 pts) - ReadPort 
> Create a `ReadPort` bundle with a 3 fields, `rAddr`, `rEn`, and `rData` that represents a single read port using parameters from `MemParams p` and IO directions that make sense. The `Data` type of `rData` will be of type `Bits`.

In [ ]:
class ReadPort(val p: MemParams) extends Bundle {
  // YOUR CODE HERE
  ???
  override def cloneType = (new ReadPort(p)).asInstanceOf[this.type]
}

In [ ]:
val p0 = MemParams(32, 64, 1, 0)
val rp0 = new ReadPort(p0) 
val elems0 = rp0.elements
assert(elems0("rEn").toString == "Bool")
assert(elems0("rData").toString == "UInt<64>")
assert(elems0("rAddr").toString == "UInt<5>")

val p1 = MemParams(128, 32, 2, 1)
val rp1 = new ReadPort(p1) 
val elems1 = rp1.elements
assert(elems1("rEn").toString == "Bool")
assert(elems1("rData").toString == "UInt<32>")
assert(elems1("rAddr").toString == "UInt<7>")

## Problem 3 (3 pts) - WritePort
> Create a `WritePort` bundle with a 3 fields, `wAddr`, `wEn`, and `wData` that represents a single write port using parameters from `MemParams p` and IO directions that make sense. The `Data` type of `wData` will be of type `Bits`.

In [ ]:
class WritePort(val p: MemParams) extends Bundle {
  // YOUR CODE HERE
  ???
  override def cloneType = (new WritePort(p)).asInstanceOf[this.type]
}

In [ ]:
val p0 = MemParams(32, 64, 0, 1)
val wp0 = new WritePort(p0) 
val elems0 = wp0.elements
assert(elems0("wEn").toString == "Bool")
assert(elems0("wData").toString == "UInt<64>")
assert(elems0("wAddr").toString == "UInt<5>")

val p1 = MemParams(128, 32, 2, 2)
val wp1 = new WritePort(p1) 
val elems1 = wp1.elements
assert(elems1("wEn").toString == "Bool")
assert(elems1("wData").toString == "UInt<32>")
assert(elems1("wAddr").toString == "UInt<7>")

## Problem 4 (5 pts) - MemIO
> Complete the `MemIO` bundle that has two fields `rports` and `wports`. Use types `Option[Vec[ReadPort]]` and `Option[Vec[WritePort]]` so the ports are pruned if there are 0 ports in that direction.

In [ ]:
class MemIO(p: MemParams) extends Bundle {
    // YOUR CODE HERE
    ???
    override def cloneType = (new MemIO(p)).asInstanceOf[this.type]
}

In [ ]:
val p0 = MemParams(32, 64, 1, 1)
val mio0 = new MemIO(p0)
val elems0 = mio0.elements
assert(elems0("rports").toString == "ReadPort[1]")
assert(elems0("wports").toString == "WritePort[1]")


val p1 = MemParams(32, 64, 2, 0)
val mio1 = new MemIO(p1)
val elems1 = mio1.elements
assert(elems1("rports").toString == "ReadPort[2]")
println(elems0("wports").toString)

val p2 = MemParams(32, 64, 0, 8)
val mio2 = new MemIO(p2)
mio2.elements
val elems2 = mio2.elements
assert(elems2("wports").toString == "WritePort[8]")


## Problem 5 (10 pts) - MultiPortMem
> Write a module that uses `MemParams`, `MemIO`, and `SyncReadMem` to create a synchronous memory with parameterizable ports and capacity.


In [ ]:
class MultiPortMem(p: MemParams) extends Module {
    val io = IO(new MemIO(p))
    // YOUR CODE HERE
    ???
}

In [ ]:
def testMultiPortMem(p: MemParams): Boolean = {
    test(new MultiPortMem(p)) { dut =>
        
        for (a <- 0 until p.numWords by p.totalPorts) { 
            // Send write req
            for (w <- 0 until p.numWritePorts) {
                dut.io.wports.get(w).wEn.poke(true.B)
                dut.io.wports.get(w).wAddr.poke((a + w).U)
                dut.io.wports.get(w).wData.poke((a + w).U)
            }
            dut.clock.step()
            
            // Send read req
            for (r <- 0 until p.numReadPorts) {
                dut.io.rports.get(r).rEn.poke(true.B)
                dut.io.rports.get(r).rAddr.poke((a + r).U)
            }
            
            dut.clock.step()
            
            // Check read req
            for (r <- 0 until p.numReadPorts) {
                dut.io.rports.get(r).rEn.poke(false.B) // turn off for next cycle
                dut.io.rports.get(r).rData.expect((a + r).U)
            }
        }
    }
    true
}

val p0 = MemParams(32, 64, 1, 1)
assert(testMultiPortMem(p0), "fail p0")

val p1 = MemParams(16, 32, 2, 2)
assert(testMultiPortMem(p1), "fail p1")

val p2 = MemParams(32, 64, 4, 4)
assert(testMultiPortMem(p2), "fail p2")